# read create qnr df from json

In [ ]:
import pandas as pd
import json

def get_gnr(qnr_path):
    with open(os.path.join(qnr_path, 'Questionnaire/content/document.json')) as file:
        json_data = json.load(file)

    question_data = []
    question_counter = 1

    def process_json_structure(children, parent_group_title):
        global question_counter
        for child in children:
            if "$type" in child:
                question_data.append({
                    "question_seq": question_counter,
                    "VariableName": child.get("VariableName"),
                    "type": child["$type"],
                    "QuestionType": child.get("QuestionType"),
                    "Answers": child.get("Answers"),
                    "Children": child.get("Children"),
                    "ConditionExpression": child.get("ConditionExpression"),
                    "HideIfDisabled": child.get("HideIfDisabled"),
                    "Featured": child.get("Featured"),
                    "Instructions": child.get("Instructions"),
                    "Properties": child.get("Properties"),
                    "PublicKey": child.get("PublicKey"),
                    "QuestionScope": child.get("QuestionScope"),
                    "QuestionText": child.get("QuestionText"),
                    "StataExportCaption": child.get("StataExportCaption"),
                    "VariableLabel": child.get("VariableLabel"),
                    "IsTimestamp": child.get("IsTimestamp"),
                    "ValidationConditions": child.get("ValidationConditions"),
                    "YesNoView": child.get("YesNoView"),
                    "Title": child.get("Title"),
                    "IsRoster": child.get("IsRoster"),
                    "parents": parent_group_title
                })
                question_counter += 1

            if "Children" in child:
                child_group_title = child.get("Title", "")
                process_json_structure(child["Children"], parent_group_title + " > " + child_group_title)

    process_json_structure(json_data["Children"], "")

    qnr_df = pd.DataFrame(question_data)
    qnr_df.loc[qnr_df['YesNoView']==True, 'type'] = 'YesNoQuestion' # create type for YesNoQuestions
    qnr_df['parents'] = qnr_df['parents'].str.lstrip(' > ')
    split_columns = qnr_df['parents'].str.split(' > ', expand=True)
    split_columns.columns = [f"parent_{i+1}" for i in range(split_columns.shape[1])]
    qnr_df = pd.concat([qnr_df, split_columns], axis=1)

    return qnr_df